## Astronomical Region Notebook
This notebook illustrates how to retrieve images of arbitrary-sized regions of the sky from on-line image databases, overlay a list of sources (in this case from SIMBAD) and load this into an interactive image/catalog viewer.

## Setup
Import some packages and define a utility function.

In [1]:
# Allways run this cell.

import os
import urllib.request
from astropy.io import ascii


# General function for retrieving URL to file

def retrieve_url_to_file(url, filename):
    
    try:
        urllib.request.urlretrieve(url, filename)
        print(f'Successfully downloaded {url} to {filename}')
    except Exception as e:
        print(f'ERROR: {e}')

## Parameters and Object Lookup

In [2]:
from astroquery.simbad import Simbad

object    = 'Messier 17'
radius    = 0.5
workspace = ('~/data/' + object).replace(' ', '_')

workspace = os.path.expanduser(workspace)

result = Simbad.query_object(object)

ra  = result[0]['ra']
dec = result[0]['dec']

print('object:    ', object)
print('region:    ', ra, dec, radius)
print('workspace: ', workspace)

try:
    os.makedirs(workspace + '/fits')
    os.makedirs(workspace + '/tbl')

except FileExistsError as e:
    print('ERROR:', str(e))
    

object:     Messier 17
region:     275.1958333333333 -16.171666666666663 0.5
workspace:  /Users/jcg/data/Messier_17
ERROR: [Errno 17] File exists: '/Users/jcg/data/Messier_17/fits'


## HiPS Map Cutouts

HiPS maps provide a heirarchy of resolutions and some of them cover the whole sky at reasonably high maximum resolutions (for instance, 2MASS with three near infrared wavelengths at 1 arcsecond).  Using a cutout service at CDS, we can extract data that can be turned into a color composite suitable for interactive visualization.

In [3]:

url = 'http://alasky.cds.unistra.fr/hips-image-services/hips2fits'
url = url + '?width=1500&height=1500'
url = url + '&projection=TAN'
url = url + '&fov=' + str(2*radius)
url = url + '&ra=' + str(ra)
url = url + '&dec=' + str(dec)


# J band

imgurl = url + '&hips=CDS/P/2MASS/J'

file = '2MASSJ.fits'

fimg = os.path.expanduser(workspace) + '/fits/' + file

retrieve_url_to_file(imgurl, fimg)


# H band

imgurl = url + '&hips=CDS/P/2MASS/H'

file = '2MASSH.fits'

fimg = os.path.expanduser(workspace) + '/fits/' + file

retrieve_url_to_file(imgurl, fimg)


# K band

imgurl = url + '&hips=CDS/P/2MASS/K'

file = '2MASSK.fits'

fimg = os.path.expanduser(workspace) + '/fits/' + file

retrieve_url_to_file(imgurl, fimg)


Successfully downloaded http://alasky.cds.unistra.fr/hips-image-services/hips2fits?width=1500&height=1500&projection=TAN&fov=1.0&ra=275.1958333333333&dec=-16.171666666666663&hips=CDS/P/2MASS/J to /Users/jcg/data/Messier_17/fits/2MASSJ.fits
Successfully downloaded http://alasky.cds.unistra.fr/hips-image-services/hips2fits?width=1500&height=1500&projection=TAN&fov=1.0&ra=275.1958333333333&dec=-16.171666666666663&hips=CDS/P/2MASS/H to /Users/jcg/data/Messier_17/fits/2MASSH.fits
Successfully downloaded http://alasky.cds.unistra.fr/hips-image-services/hips2fits?width=1500&height=1500&projection=TAN&fov=1.0&ra=275.1958333333333&dec=-16.171666666666663&hips=CDS/P/2MASS/K to /Users/jcg/data/Messier_17/fits/2MASSK.fits


## SIMBAD Region Search

Frequently, it helps to compare the image with source lists. There are thousands of these available via TAP (Table Access Protocol) services. But rather than track these down individually, here we will use SIMBAD to get a composite list and then extract subsets either based on the source name (<i>i.e.</i>, from specific catalogs) or by object type.

In [4]:
from astropy.table import Table
from astroquery.simbad import Simbad
import pandas as pd

simbad = Simbad()

simbad.add_votable_fields('otype')

simbad.ROW_LIMIT = 100000

simdata = simbad.query_region(object, radius=str(radius)+'d')

length = len(simdata)

ftbl = workspace + '/tbl/simbad.tbl'

simdata.write(ftbl, format='ipac', overwrite=True)

print(f'{length} records written to {ftbl}')

simbad_df = simdata.to_pandas()

3471 records written to /Users/jcg/data/Messier_17/tbl/simbad.tbl


## Young Stellar Objects from the SIMBAD List
The SIMBAD list for this region includes a large number of catalogs, including a few aimed at Young Stellar Objects (YSOs).  We next extract three of these: the Spitzer/IRAC Candidate YSO (SPICY) catalog; the APEX Telescope Large Area Survey of the Galaxy (AGAL); and CXOU, a collection of "unregistered" Chandra X-ray Observatory observations (occasional sources published by individuals).  We also extract a list of all young stellar objects.

In [5]:
# SPICY

spicy_df = simbad_df[simbad_df['main_id'].str.contains('SPICY')]

table = Table.from_pandas(spicy_df)

length = len(spicy_df)

ftbl = workspace + '/tbl/spicy.tbl'

table.write(ftbl, format='ipac', overwrite=True)

print(f'{length} records written to {ftbl}')


# AGAL

agal_df = simbad_df[simbad_df['main_id'].str.contains('AGAL')]

table = Table.from_pandas(agal_df)

length = len(agal_df)

ftbl = workspace + '/tbl/agal.tbl'

table.write(ftbl, format='ipac', overwrite=True)

print(f'{length} records written to {ftbl}')


# CXOU

cxou_df = simbad_df[simbad_df['main_id'].str.contains('CXOU')]

table = Table.from_pandas(cxou_df)

length = len(cxou_df)

ftbl = workspace + '/tbl/cxou.tbl'

table.write(ftbl, format='ipac', overwrite=True)

print(f'{length} records written to {ftbl}')



# Young Stellar Objects by Type

yso_df = simbad_df[simbad_df['otype'].str.contains('Y*O')]

table = Table.from_pandas(yso_df)

length = len(yso_df)

ftbl = workspace + '/tbl/yso.tbl'

table.write(ftbl, format='ipac', overwrite=True)

print(f'{length} records written to {ftbl}')



64 records written to /Users/jcg/data/Messier_17/tbl/spicy.tbl
56 records written to /Users/jcg/data/Messier_17/tbl/agal.tbl
501 records written to /Users/jcg/data/Messier_17/tbl/cxou.tbl
1003 records written to /Users/jcg/data/Messier_17/tbl/yso.tbl


### Defining the Layout of the Display

Unless we tell it, there is no way for the viewer to decide what data from the workspace to display, how to stretch the image (or images in the case of 3-color data), what symbols to use for point source catalogs (and their scaling an color), whether to show coordinate grids, and so on.

Some displays can be handled with defaults but this one is a little too complicated for that to work well.  So we will go the other route here of defining everything ourselves through a JSON layout file. For more information on mViewer, see http://montage.ipac.caltech.edu/mViewer.

In this case, we are not going to show the YSO table initially in favor of just showing the subsets from specific catalogs but you are free to change that.

In [6]:
import json

layout = {
  "Image Label": object + ' (' + str(radius) + ' degrees)',

  "Color":
  [
    {
      "Color": "Blue",
      "File": "fits/2MASSJ.fits",
      "Stretch Mode": "Gaussian-log",
      "Min":  "-0.5", "Min Units": "sigma",
      "Max": "max", "Max Units": "sigma"
    },

    {
      "Color": "Green",
      "File": "fits/2MASSH.fits",
      "Stretch Mode": "Gaussian-log",
      "Min":  "-0.5", "Min Units": "sigma",
      "Max": "max", "Max Units": "sigma"
    },

    {
      "Color": "Red",
      "File": "fits/2MASSK.fits",
      "Stretch Mode": "Gaussian-log",
      "Min":  "-0.5", "Min Units": "sigma",
      "Max": "max", "Max Units": "sigma"
    }
  ],


  "Overlays":
  [
    {"Type": "Catalog", "Color": "ffa0a0", "File": "tbl/spicy.tbl", "Size of Ref Value": 2, "Symbol": "box"},
    {"Type": "Catalog", "Color": "00ffff", "File": "tbl/agal.tbl",  "Size of Ref Value": 2, "Symbol": "circle"},
    {"Type": "Catalog", "Color": "ffffff",  "File": "tbl/cxou.tbl",  "Size of Ref Value": 1, "Symbol": "plus"},

    {"Type": "Eq grid", "Color": "lightgray"}
  ]
}


filename = workspace + "/layout.json"

with open(filename, 'w') as file:
    json.dump(layout, file, indent=2)

print("JSON layout written to", filename)

JSON layout written to /Users/jcg/data/Messier_17/layout.json


## Region Vizualization

Please read this description before running the next cell.  The 'koaViewer' app helps you visualize the data you have downloaded into the workspace.  This workspace contains a 'tbl' subdirectory with all the table files you have downloaded (both source tables and image metadata tables) and a 'fits' directory with a FITS image file or files.  The data can easily get too involved for a single simple display (though what we have done through this Notebook is simple enough), but even here someone has to specify things like which tables to actually display, the colors of overlays in the image display, symbol sizes, <i>etc.</i>

So the actual image display gets defined by a JSON structure file ('layout.json') in the top level of the workspace.  

There are three ways to build this JSON.  If no layout.json exists when koaViewer starts up, it will analyze the contents of the workspace and take a guess at the structure using a few simple rules.  This is likely to be sub-optimal.  Alternatively, knowing what we have done in the Notebook to construct the workspace, we can construct the JSON manually (<i>i.e.,</i> setting up the JSON in a Notebook cell).  Finally, once we have a starting point, the koaViewer application has layout editing pop-ups where we can fine tune the details (like color and column-base symbol sizes).

Any changes to the this Notebook (such as the location/search radius in the first step) can affect how many tables there are to display, hard-coded JSON is probably inadequate, so we will go with first option and let koaViewer define the JSON we start with.

To start koaViewer, run the next cell and activate the link it produces.

In [7]:
from koaviewer import koaViewer

viewer = koaViewer()

viewer.run(workspace)

Dash app running on http://127.0.0.1:54593/
